# googlenet


rcnn
데이터가 모자랄땐 pretrain모델을 사용해보는게 좋다
-----

22layers deep network 깊어진 레이어 

1x1 conv 사용

concatenated = 붙인거 ..




In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt


import torch.optim as optim
from torchsummary import summary
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

In [2]:
class Conv_Block(nn.Module):
    def __init__(self,in_channels, out_channels,**kwargs) :
        super(Conv_Block, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()



    def forward(self, x):
        x = self.conv(x)
        x = self.batch_norm(x)
        x = self.relu(x)


        return x





In [3]:
Conv_Block(3,64,kernel_size =1, stride =1, padding=0)

Conv_Block(
  (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
  (batch_norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)

In [4]:
class Inception(nn.Module):
    def __init__(self, in_channels, n1x1, n3x3_reduce, n3x3, n5x5_reduce, n5x5, pool_proj): 
        # n3x3_reduce는 3*3 conv전 1*1conv의 출력, 채널 reduce를 위해 사용
        # pool_proj 마지막 풀링하고 1*1 하는거 . 프로젝션이라고 함
        super(Inception, self).__init__()

        self.branch_1 = Conv_Block(in_channels, n1x1, kernel_size = 1, stride=1, padding=0)

        self.branch_2 = nn.Sequential(Conv_Block(in_channels, n3x3_reduce, kernel_size=1, stride=1, padding =0),
                                        Conv_Block(n3x3_reduce, n3x3, kernel_size =1, stride=1, padding=0))

        self.branch_3 = nn.Sequential(Conv_Block(in_channels,n5x5_reduce,kernel_size=1, stride=1, padding =0),
                                        Conv_Block(n5x5_reduce,n5x5, kernel_size=5, stride=1, padding =2))

        self.branch_4 = nn.Sequential(nn.MaxPool2d(kernel_size=3, stride=1, padding=1), # 맥스풀링은 피처맵사이즈 만 줄이니까 ,채널에 관한거는 인채널 아웃채널 업슴
                                        Conv_Block(in_channels,pool_proj, kernel_size=1, stride=1, padding=0))


    def forward(self, x):
        x_1 = self.branch_1(x)
        x_2 = self.branch_2(x)
        x_3 = self.branch_3(x)
        x_4 = self.branch_4(x)

        return torch.cat([x_1,x_2,x_3,x_4], dim=1)
        # 여기서는 채널을 합친다는 의미로 1이 된다. 그래서 feature맵 크기가 같아야한다 ! 그래서 위에서 f =f 다 하고 생각하면서 계산을했군요. 
        # feature맵 사이즈를 유지시켰음음 그게 concat하기 위해서였던것임


In [5]:
# b,c,h,w, 순서
### depth = channel 여기서 depth는 채널

# 모델의 depth = 깊이 레이어의 깊이
# with를 키운다 = 채널을 키운다 고하네요

In [6]:
class Inception_Aux(nn.Module):
    def __init__(self, in_channels):
        super(Inception_Aux,self).__init__()
        self.avg_pool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv = Conv_Block(in_channels, 128, kernel_size =1, stride=1, padding=0)
        self.fc1 = nn.Linear(in_features=128* 4*4, out_features=1024) # 2048은 128* 4*4
        self.fc2 = nn.Linear(in_features=1024, out_features=10)
    

    def forward(self,x):
        x = self.avg_pool(x)
        x = self.conv(x)

        x= x.view(-1, 128*4*4) # 1d tensor로 만들어주기

        x = F.dropout(F.relu(self.fc1(x)), p=0.7)
        x = self.fc2(x)

        return x

In [7]:
Inception_Aux(512)

Inception_Aux(
  (avg_pool): AvgPool2d(kernel_size=5, stride=3, padding=0)
  (conv): Conv_Block(
    (conv): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
    (batch_norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

In [8]:
i_m = Inception(192,64,96,128,16,32,32)

In [9]:
summary(i_m, (192,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]          12,352
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
        Conv_Block-4         [-1, 64, 224, 224]               0
            Conv2d-5         [-1, 96, 224, 224]          18,528
       BatchNorm2d-6         [-1, 96, 224, 224]             192
              ReLU-7         [-1, 96, 224, 224]               0
        Conv_Block-8         [-1, 96, 224, 224]               0
            Conv2d-9        [-1, 128, 224, 224]          12,416
      BatchNorm2d-10        [-1, 128, 224, 224]             256
             ReLU-11        [-1, 128, 224, 224]               0
       Conv_Block-12        [-1, 128, 224, 224]               0
           Conv2d-13         [-1, 16, 224, 224]           3,088
      BatchNorm2d-14         [-1, 16, 2

In [10]:
i_a = Inception_Aux(512)

In [11]:
summary(i_a,(512,14,14))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         AvgPool2d-1            [-1, 512, 4, 4]               0
            Conv2d-2            [-1, 128, 4, 4]          65,664
       BatchNorm2d-3            [-1, 128, 4, 4]             256
              ReLU-4            [-1, 128, 4, 4]               0
        Conv_Block-5            [-1, 128, 4, 4]               0
            Linear-6                 [-1, 1024]       2,098,176
            Linear-7                   [-1, 10]          10,250
Total params: 2,174,346
Trainable params: 2,174,346
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.38
Forward/backward pass size (MB): 0.13
Params size (MB): 8.29
Estimated Total Size (MB): 8.81
----------------------------------------------------------------


In [15]:
class GoogleNet(nn.Module):
    def __init__(self, train_):
        super(GoogleNet, self).__init__()
        self.training = train_

        self.conv1= nn.Conv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64,kernel_size=1, stride=1, padding=0)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=192, kernel_size=3, stride=1 ,padding =1)
        self.inc_3a = Inception(192, 64, 96, 128, 16, 32, 32) # 64+128+32+32
        self.inc_3b = Inception(256, 128, 128, 192, 32, 96, 64)
        
        self.inc_4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inc_4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inc_4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inc_4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inc_4e = Inception(528, 256, 160, 320, 32, 128, 128)

        self.inc_5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inc_5b = Inception(832, 384, 192, 384, 48, 128, 128)

        self.aux_1 = Inception_Aux(512)
        self.aux_2 = Inception_Aux(528)

        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding =1)
        self.avg_pool = nn.AvgPool2d(kernel_size=7, stride=1) # 에버리지풀링은 이미지 사이즈1*!로 만드는거라 패딩업슴

        self.fc1 = nn.Linear(in_features=1024, out_features=10)

        self.lrn = nn.LocalResponseNorm(2)

    def forward(self,x):
        #
        x = self.lrn(self.max_pool(F.relu(self.conv1(x))))

        #
        x = F.relu(self.conv2(x))
        x = self.max_pool(self.lrn(F.relu(self.conv3(x))))
       
        #inception 3.
        x = self.inc_3a(x)
        x = self.inc_3b(x)
        x = self.max_pool(x)

        #inception 4.
        x = self.inc_4a(x)

        if self.training:
            loss_1 = self.aux_1(x)
        
        x = self.inc_4b(x)
        x = self.inc_4c(x)
        x = self.inc_4d(x)

        if self.training:
            loss_2 = self.aux_2(x)

        x = self.inc_4e(x)
        x = self.max_pool(x)

        # inception 5.
        x = self.inc_5a(x)
        x = self.inc_5b(x)

        x = self.avg_pool(x)

        x = x.view(-1,1024)

        x = F.dropout(x, p=0.4)
        x = self.fc1(x)

        if self.training:
            return x, loss_1, loss_2
        else:
            return x

In [16]:
model = GoogleNet(train_=True)

In [17]:
summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
         MaxPool2d-2           [-1, 64, 56, 56]               0
 LocalResponseNorm-3           [-1, 64, 56, 56]               0
            Conv2d-4           [-1, 64, 56, 56]           4,160
            Conv2d-5          [-1, 192, 56, 56]         110,784
 LocalResponseNorm-6          [-1, 192, 56, 56]               0
         MaxPool2d-7          [-1, 192, 28, 28]               0
            Conv2d-8           [-1, 64, 28, 28]          12,352
       BatchNorm2d-9           [-1, 64, 28, 28]             128
             ReLU-10           [-1, 64, 28, 28]               0
       Conv_Block-11           [-1, 64, 28, 28]               0
           Conv2d-12           [-1, 96, 28, 28]          18,528
      BatchNorm2d-13           [-1, 96, 28, 28]             192
             ReLU-14           [-1, 96,